# Paper 1: Cross-Architecture Knowledge Transfer via HDC
## Experiment 1: Complete Transfer Pipeline

This notebook implements the complete experimental pipeline:
- **Stage 0**: Model ceilings (fine-tuned baselines)
- **Stage 1**: HDC quantization cost
- **Stage 2**: Alignment methods comparison
- **Stage 3**: Model pairs (bidirectional transfer)
- **Stage 4**: Task generalization (SST-2, AG News)

**Author**: Nikolay Yudin  
**Project**: Semantic Event Protocol (SEP)  
**Repository**: https://github.com/nick-yudin/SEP

## Setup

In [ ]:
# Install dependencies
!pip install -q transformers datasets torch numpy scikit-learn matplotlib tqdm

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.cross_decomposition import CCA
from scipy.linalg import orthogonal_procrustes
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import gc
import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

In [ ]:
# Configuration
CONFIG = {
    'models': {
        'distilbert': 'distilbert-base-uncased',
        'gpt2': 'gpt2',
        'roberta': 'roberta-base',
    },
    'datasets': ['sst2', 'ag_news'],  # No MNLI - requires different encoding
    'hdc_dims': [1024, 2048, 4096, 8192],
    'anchor_sizes': [100, 500, 1000],
    'tau': 0.3,  # Ternary threshold
    'seeds': [42, 123, 456],
    'train_size': 3000,
    'test_size': 500,
    'anchor_pool_size': 2000,
    'classifier_epochs': 20,
    'contrastive_epochs': 20,
    'finetune_epochs': 3,
}

print('Configuration loaded')

## Core Classes

In [ ]:
class EmbeddingExtractor:
    """Extract embeddings from transformer models using mean pooling."""
    
    def __init__(self, model_name, device='cuda'):
        self.model_name = model_name
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.model.eval()
        
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.embed_dim = self.model.config.hidden_size
        print(f'Loaded {model_name}: {self.embed_dim}d')
    
    def encode(self, texts, batch_size=32, show_progress=True):
        """Extract embeddings using mean pooling."""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        iterator = range(0, len(texts), batch_size)
        if show_progress:
            iterator = tqdm(iterator, desc=f'Encoding')
        
        for i in iterator:
            batch = texts[i:i+batch_size]
            inputs = self.tokenizer(
                batch, padding=True, truncation=True,
                max_length=128, return_tensors='pt'
            ).to(self.device)
            
            with torch.no_grad():
                outputs = self.model(**inputs)
                hidden = outputs.last_hidden_state
                mask = inputs['attention_mask'].unsqueeze(-1)
                embeddings = (hidden * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
            
            all_embeddings.append(embeddings.cpu().numpy())
        
        return np.vstack(all_embeddings)
    
    def clear(self):
        """Free GPU memory."""
        del self.model
        del self.tokenizer
        torch.cuda.empty_cache()
        gc.collect()

print('EmbeddingExtractor ready')

In [ ]:
class HDCEncoder:
    """Encode float embeddings to ternary HDC vectors."""
    
    def __init__(self, input_dim, hdc_dim, tau=0.3, seed=42):
        self.input_dim = input_dim
        self.hdc_dim = hdc_dim
        self.tau = tau
        
        np.random.seed(seed)
        self.projection = np.random.randn(input_dim, hdc_dim).astype(np.float32)
        self.projection /= np.linalg.norm(self.projection, axis=0, keepdims=True)
    
    def encode(self, embeddings):
        """Project and quantize to ternary."""
        projected = embeddings @ self.projection
        std = np.std(projected, axis=1, keepdims=True)
        threshold = self.tau * std
        
        ternary = np.zeros_like(projected, dtype=np.int8)
        ternary[projected > threshold] = 1
        ternary[projected < -threshold] = -1
        
        return ternary
    
    def encode_float(self, embeddings):
        """Project without quantization (for alignment training)."""
        return embeddings @ self.projection

print('HDCEncoder ready')

In [ ]:
class ContrastiveAligner(nn.Module):
    """Learn alignment between teacher and student embedding spaces."""
    
    def __init__(self, teacher_dim, student_dim, hidden_dim=512):
        super().__init__()
        self.teacher_proj = nn.Sequential(
            nn.Linear(teacher_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
        )
        self.student_proj = nn.Sequential(
            nn.Linear(student_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
        )
    
    def forward(self, teacher_emb, student_emb):
        t_proj = nn.functional.normalize(self.teacher_proj(teacher_emb), dim=-1)
        s_proj = nn.functional.normalize(self.student_proj(student_emb), dim=-1)
        return t_proj, s_proj


def train_contrastive_aligner(teacher_emb, student_emb, hidden_dim=512, epochs=20, lr=1e-3, device='cuda'):
    """Train contrastive alignment between embedding spaces."""
    teacher_dim = teacher_emb.shape[1]
    student_dim = student_emb.shape[1]
    
    aligner = ContrastiveAligner(teacher_dim, student_dim, hidden_dim).to(device)
    optimizer = torch.optim.Adam(aligner.parameters(), lr=lr)
    
    T = torch.tensor(teacher_emb, dtype=torch.float32).to(device)
    S = torch.tensor(student_emb, dtype=torch.float32).to(device)
    
    batch_size = min(256, len(teacher_emb))
    n_batches = len(teacher_emb) // batch_size
    
    for epoch in range(epochs):
        perm = torch.randperm(len(teacher_emb))
        total_loss = 0
        
        for i in range(n_batches):
            idx = perm[i*batch_size:(i+1)*batch_size]
            t_batch = T[idx]
            s_batch = S[idx]
            
            t_proj, s_proj = aligner(t_batch, s_batch)
            
            # Contrastive loss: positive pairs should be similar
            pos_sim = (t_proj * s_proj).sum(dim=-1)
            
            # Negative pairs: shifted indices
            neg_idx = torch.roll(torch.arange(len(idx)), 1)
            neg_sim = (t_proj * s_proj[neg_idx]).sum(dim=-1)
            
            loss = -pos_sim.mean() + torch.clamp(neg_sim + 0.5, min=0).mean()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
    
    # Compute final alignment similarity
    aligner.eval()
    with torch.no_grad():
        t_proj, s_proj = aligner(T, S)
        similarity = (t_proj * s_proj).sum(dim=-1).mean().item()
    
    return aligner, similarity

print('ContrastiveAligner ready')

## Data Loading

In [ ]:
def load_classification_data(dataset_name, train_size, test_size, anchor_size, seed=42):
    """Load and prepare classification dataset."""
    np.random.seed(seed)
    
    if dataset_name == 'sst2':
        dataset = load_dataset('glue', 'sst2')
        texts = dataset['train']['sentence']
        labels = dataset['train']['label']
        num_classes = 2
    elif dataset_name == 'ag_news':
        dataset = load_dataset('ag_news')
        texts = dataset['train']['text']
        labels = dataset['train']['label']
        num_classes = 4
    else:
        raise ValueError(f'Unknown dataset: {dataset_name}')
    
    # Shuffle and split
    indices = np.random.permutation(len(texts))
    total_needed = train_size + test_size + anchor_size
    indices = indices[:total_needed]
    
    texts = [texts[i] for i in indices]
    labels = [labels[i] for i in indices]
    
    return {
        'train_texts': texts[:train_size],
        'train_labels': labels[:train_size],
        'test_texts': texts[train_size:train_size+test_size],
        'test_labels': labels[train_size:train_size+test_size],
        'anchor_texts': texts[train_size+test_size:],
        'num_classes': num_classes,
    }

print('Data loading ready')

## Stage 0: Model Ceilings

In [ ]:
def compute_finetuned_ceiling(model_name, dataset_name, seed=42):
    """Fine-tune model and compute ceiling accuracy."""
    from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
    import evaluate
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    # Load data
    if dataset_name == 'sst2':
        dataset = load_dataset('glue', 'sst2')
        num_labels = 2
        text_key = 'sentence'
    else:
        dataset = load_dataset('ag_news')
        num_labels = 4
        text_key = 'text'
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=num_labels
    )
    if model.config.pad_token_id is None:
        model.config.pad_token_id = tokenizer.pad_token_id
    
    def tokenize(examples):
        return tokenizer(examples[text_key], truncation=True, padding=True, max_length=128)
    
    train_data = dataset['train'].shuffle(seed=seed).select(range(3000)).map(tokenize, batched=True)
    test_data = dataset['test' if 'test' in dataset else 'validation'].shuffle(seed=seed).select(range(500)).map(tokenize, batched=True)
    
    training_args = TrainingArguments(
        output_dir='./tmp_trainer',
        num_train_epochs=CONFIG['finetune_epochs'],
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        logging_steps=100,
        save_strategy='no',
        report_to='none',
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=test_data,
    )
    
    trainer.train()
    preds = trainer.predict(test_data)
    accuracy = (preds.predictions.argmax(-1) == preds.label_ids).mean()
    
    # Cleanup
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    
    return accuracy

print('Stage 0 ready')

In [ ]:
# Run Stage 0
print('=' * 60)
print('STAGE 0: Model Ceilings (Fine-tuned)')
print('=' * 60)

stage0_results = {}
for model_key, model_name in CONFIG['models'].items():
    print(f'\nFine-tuning {model_key}...')
    acc = compute_finetuned_ceiling(model_name, 'sst2')
    stage0_results[model_key] = acc
    print(f'  {model_key}: {acc:.1%}')

print('\nStage 0 complete!')

## Stage 1: HDC Quantization Cost

In [ ]:
def compute_hdc_accuracy(embeddings, labels, hdc_dim, tau=0.3, seed=42):
    """Compute accuracy using HDC ternary vectors."""
    np.random.seed(seed)
    
    # Split
    n_train = int(0.8 * len(embeddings))
    train_emb, test_emb = embeddings[:n_train], embeddings[n_train:]
    train_labels, test_labels = labels[:n_train], labels[n_train:]
    
    # HDC encode
    hdc = HDCEncoder(embeddings.shape[1], hdc_dim, tau, seed)
    train_hdc = hdc.encode(train_emb)
    test_hdc = hdc.encode(test_emb)
    
    # Train classifier
    clf = LogisticRegression(max_iter=1000, random_state=seed)
    clf.fit(train_hdc, train_labels)
    
    return clf.score(test_hdc, test_labels)

print('Stage 1 functions ready')

In [ ]:
# Run Stage 1
print('=' * 60)
print('STAGE 1: HDC Quantization Cost')
print('=' * 60)

# Load SST-2 data
data = load_classification_data('sst2', CONFIG['train_size'], CONFIG['test_size'], 0)
all_texts = data['train_texts'] + data['test_texts']
all_labels = data['train_labels'] + data['test_labels']

stage1_results = {}

for model_key, model_name in list(CONFIG['models'].items())[:2]:  # distilbert, gpt2
    print(f'\n{model_key}:')
    
    extractor = EmbeddingExtractor(model_name, device)
    embeddings = extractor.encode(all_texts)
    
    stage1_results[model_key] = {
        'embed_dim': extractor.embed_dim,
        'hdc_results': {}
    }
    
    for hdc_dim in CONFIG['hdc_dims']:
        acc = compute_hdc_accuracy(embeddings, all_labels, hdc_dim)
        stage1_results[model_key]['hdc_results'][hdc_dim] = acc
        print(f'  HDC {hdc_dim}d: {acc:.1%}')
    
    extractor.clear()

print('\nStage 1 complete!')

## Stage 2: Alignment Methods

In [ ]:
def run_alignment_experiment(teacher_emb, student_emb, train_labels, test_labels,
                            anchor_teacher, anchor_student, method, hdc_dim=4096, seed=42):
    """Run single alignment experiment."""
    np.random.seed(seed)
    
    result = {'method': method, 'seed': seed}
    
    # HDC encode
    hdc = HDCEncoder(teacher_emb.shape[1], hdc_dim, CONFIG['tau'], seed)
    
    if method == 'none':
        # No alignment - just use same projection
        hdc_student = HDCEncoder(student_emb.shape[1], hdc_dim, CONFIG['tau'], seed)
        train_hdc = hdc.encode(teacher_emb)
        test_hdc = hdc_student.encode(student_emb)
        
    elif method == 'procrustes':
        # Procrustes alignment
        R, _ = orthogonal_procrustes(anchor_teacher, anchor_student)
        result['align_error'] = np.linalg.norm(anchor_teacher @ R - anchor_student)
        
        aligned_teacher = teacher_emb @ R
        train_hdc = hdc.encode(aligned_teacher)
        
        hdc_student = HDCEncoder(student_emb.shape[1], hdc_dim, CONFIG['tau'], seed)
        test_hdc = hdc_student.encode(student_emb)
        
    elif method == 'cca':
        # CCA alignment
        n_components = min(256, anchor_teacher.shape[0] - 1, anchor_teacher.shape[1], anchor_student.shape[1])
        cca = CCA(n_components=n_components)
        cca.fit(anchor_teacher, anchor_student)
        
        teacher_cca = cca.transform(teacher_emb)
        student_cca = cca.transform(student_emb)
        
        result['n_components'] = n_components
        result['correlation'] = np.corrcoef(teacher_cca[:, 0], student_cca[:, 0])[0, 1]
        
        hdc_cca = HDCEncoder(n_components, hdc_dim, CONFIG['tau'], seed)
        train_hdc = hdc_cca.encode(teacher_cca)
        test_hdc = hdc_cca.encode(student_cca)
        
    elif method == 'contrastive':
        # Contrastive alignment
        aligner, similarity = train_contrastive_aligner(
            anchor_teacher, anchor_student,
            hidden_dim=512, epochs=CONFIG['contrastive_epochs'], device=device
        )
        result['similarity'] = similarity
        
        # Project through aligner
        aligner.eval()
        with torch.no_grad():
            T = torch.tensor(teacher_emb, dtype=torch.float32).to(device)
            S = torch.tensor(student_emb, dtype=torch.float32).to(device)
            teacher_proj, _ = aligner(T, T)  # Just need teacher projection
            _, student_proj = aligner(S, S)  # Just need student projection
            
            teacher_aligned = teacher_proj.cpu().numpy()
            student_aligned = student_proj.cpu().numpy()
        
        hdc_aligned = HDCEncoder(teacher_aligned.shape[1], hdc_dim, CONFIG['tau'], seed)
        train_hdc = hdc_aligned.encode(teacher_aligned)
        test_hdc = hdc_aligned.encode(student_aligned)
    
    # Train and evaluate
    clf = LogisticRegression(max_iter=1000, random_state=seed)
    clf.fit(train_hdc, train_labels)
    result['accuracy'] = clf.score(test_hdc, test_labels)
    
    return result

print('Stage 2 functions ready')

In [ ]:
# Run Stage 2
print('=' * 60)
print('STAGE 2: Alignment Methods Comparison')
print('=' * 60)

# Load data with anchors
data = load_classification_data('sst2', CONFIG['train_size'], CONFIG['test_size'], CONFIG['anchor_pool_size'])

# Extract embeddings
teacher_name = CONFIG['models']['distilbert']
student_name = CONFIG['models']['gpt2']

print(f'\nTeacher: {teacher_name}')
teacher_ext = EmbeddingExtractor(teacher_name, device)
teacher_train_emb = teacher_ext.encode(data['train_texts'])
teacher_anchor_emb = teacher_ext.encode(data['anchor_texts'])
teacher_ext.clear()

print(f'Student: {student_name}')
student_ext = EmbeddingExtractor(student_name, device)
student_test_emb = student_ext.encode(data['test_texts'])
student_anchor_emb = student_ext.encode(data['anchor_texts'])

# Compute student ceiling
student_train_emb = student_ext.encode(data['train_texts'])
hdc_student = HDCEncoder(student_train_emb.shape[1], 4096, CONFIG['tau'], 42)
clf = LogisticRegression(max_iter=1000)
clf.fit(hdc_student.encode(student_train_emb), data['train_labels'])
student_ceiling = clf.score(hdc_student.encode(student_test_emb), data['test_labels'])
print(f'\nStudent ceiling: {student_ceiling:.1%}')

student_ext.clear()

# Run experiments
stage2_results = {
    'teacher': teacher_name,
    'student': student_name,
    'student_ceiling': student_ceiling,
    'experiments': []
}

methods = ['none', 'procrustes', 'cca', 'contrastive']

for anchor_size in CONFIG['anchor_sizes']:
    print(f'\nAnchor size: {anchor_size}')
    anchor_t = teacher_anchor_emb[:anchor_size]
    anchor_s = student_anchor_emb[:anchor_size]
    
    for method in methods:
        for seed in CONFIG['seeds']:
            result = run_alignment_experiment(
                teacher_train_emb, student_test_emb,
                data['train_labels'], data['test_labels'],
                anchor_t, anchor_s, method, seed=seed
            )
            result['anchor_size'] = anchor_size
            result['efficiency'] = result['accuracy'] / student_ceiling
            stage2_results['experiments'].append(result)
            print(f'  {method} (seed={seed}): {result["accuracy"]:.1%} ({result["efficiency"]:.1%} eff)')

print('\nStage 2 complete!')

## Stage 3: Model Pairs

In [ ]:
def run_transfer_experiment(teacher_name, student_name, data, seed=42):
    """Run transfer from teacher to student."""
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    # Extract teacher embeddings
    teacher_ext = EmbeddingExtractor(teacher_name, device)
    teacher_train_emb = teacher_ext.encode(data['train_texts'])
    teacher_anchor_emb = teacher_ext.encode(data['anchor_texts'][:500])
    teacher_ext.clear()
    
    # Extract student embeddings
    student_ext = EmbeddingExtractor(student_name, device)
    student_test_emb = student_ext.encode(data['test_texts'])
    student_anchor_emb = student_ext.encode(data['anchor_texts'][:500])
    student_train_emb = student_ext.encode(data['train_texts'])
    
    # Student ceiling
    hdc = HDCEncoder(student_train_emb.shape[1], 4096, CONFIG['tau'], seed)
    clf = LogisticRegression(max_iter=1000, random_state=seed)
    clf.fit(hdc.encode(student_train_emb), data['train_labels'])
    ceiling = clf.score(hdc.encode(student_test_emb), data['test_labels'])
    
    student_ext.clear()
    
    # Train contrastive aligner
    aligner, similarity = train_contrastive_aligner(
        teacher_anchor_emb, student_anchor_emb,
        hidden_dim=512, epochs=CONFIG['contrastive_epochs'], device=device
    )
    
    # Apply alignment
    aligner.eval()
    with torch.no_grad():
        T = torch.tensor(teacher_train_emb, dtype=torch.float32).to(device)
        S = torch.tensor(student_test_emb, dtype=torch.float32).to(device)
        teacher_proj = aligner.teacher_proj(T)
        student_proj = aligner.student_proj(S)
        teacher_aligned = nn.functional.normalize(teacher_proj, dim=-1).cpu().numpy()
        student_aligned = nn.functional.normalize(student_proj, dim=-1).cpu().numpy()
    
    # HDC encode and classify
    hdc_aligned = HDCEncoder(teacher_aligned.shape[1], 4096, CONFIG['tau'], seed)
    train_hdc = hdc_aligned.encode(teacher_aligned)
    test_hdc = hdc_aligned.encode(student_aligned)
    
    clf = LogisticRegression(max_iter=1000, random_state=seed)
    clf.fit(train_hdc, data['train_labels'])
    accuracy = clf.score(test_hdc, data['test_labels'])
    
    return {
        'seed': seed,
        'accuracy': accuracy,
        'efficiency': accuracy / ceiling,
        'similarity': similarity,
        'ceiling': ceiling
    }

print('Stage 3 functions ready')

In [ ]:
# Run Stage 3
print('=' * 60)
print('STAGE 3: Model Pairs (Bidirectional Transfer)')
print('=' * 60)

data = load_classification_data('sst2', CONFIG['train_size'], CONFIG['test_size'], CONFIG['anchor_pool_size'])

model_pairs = [
    ('distilbert', 'gpt2'),
    ('gpt2', 'distilbert'),
    ('distilbert', 'roberta'),
    ('roberta', 'distilbert'),
]

stage3_results = {'pairs': []}

for teacher_key, student_key in model_pairs:
    print(f'\n{teacher_key} → {student_key}')
    
    teacher_name = CONFIG['models'][teacher_key]
    student_name = CONFIG['models'][student_key]
    
    pair_results = {
        'teacher': teacher_key,
        'student': student_key,
        'transfers': []
    }
    
    for seed in CONFIG['seeds']:
        result = run_transfer_experiment(teacher_name, student_name, data, seed)
        pair_results['transfers'].append(result)
        print(f'  seed={seed}: {result["accuracy"]:.1%} ({result["efficiency"]:.1%} eff)')
    
    # Aggregate
    pair_results['student_ceiling'] = np.mean([r['ceiling'] for r in pair_results['transfers']])
    pair_results['mean_accuracy'] = np.mean([r['accuracy'] for r in pair_results['transfers']])
    pair_results['mean_efficiency'] = np.mean([r['efficiency'] for r in pair_results['transfers']])
    
    stage3_results['pairs'].append(pair_results)

print('\nStage 3 complete!')

## Stage 4: Task Generalization

In [ ]:
# Run Stage 4
print('=' * 60)
print('STAGE 4: Task Generalization (SST-2, AG News)')
print('=' * 60)

stage4_results = {'datasets': {}}

for dataset_name in CONFIG['datasets']:
    print(f'\n--- {dataset_name.upper()} ---')
    
    data = load_classification_data(dataset_name, CONFIG['train_size'], CONFIG['test_size'], CONFIG['anchor_pool_size'])
    
    dataset_results = {
        'name': dataset_name,
        'num_classes': data['num_classes'],
        'pairs': []
    }
    
    for teacher_key, student_key in [('distilbert', 'gpt2'), ('gpt2', 'distilbert')]:
        print(f'\n{teacher_key} → {student_key}')
        
        teacher_name = CONFIG['models'][teacher_key]
        student_name = CONFIG['models'][student_key]
        
        pair_results = {
            'teacher': teacher_key,
            'student': student_key,
            'transfers': []
        }
        
        for seed in CONFIG['seeds']:
            result = run_transfer_experiment(teacher_name, student_name, data, seed)
            pair_results['transfers'].append({
                'seed': seed,
                'accuracy': result['accuracy'],
                'efficiency': result['efficiency']
            })
            print(f'  seed={seed}: {result["accuracy"]:.1%} ({result["efficiency"]:.1%} eff)')
        
        pair_results['student_ceiling'] = np.mean([r['ceiling'] for r in [result]])
        pair_results['mean_accuracy'] = np.mean([r['accuracy'] for r in pair_results['transfers']])
        pair_results['mean_efficiency'] = np.mean([r['efficiency'] for r in pair_results['transfers']])
        
        dataset_results['pairs'].append(pair_results)
    
    stage4_results['datasets'][dataset_name] = dataset_results

print('\nStage 4 complete!')

## Save Results

In [ ]:
def convert_to_serializable(obj):
    """Convert numpy types to Python types for JSON serialization."""
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(v) for v in obj]
    elif isinstance(obj, (np.float32, np.float64, np.floating)):
        return float(obj)
    elif isinstance(obj, (np.int32, np.int64, np.integer)):
        return int(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

# Compile all results
all_results = {
    'config': CONFIG,
    'stage0': stage0_results,
    'stage1': stage1_results,
    'stage2': stage2_results,
    'stage3': stage3_results,
    'stage4': stage4_results,
}

# Save
with open('paper1_experiment1_results.json', 'w') as f:
    json.dump(convert_to_serializable(all_results), f, indent=2)

print('Results saved to paper1_experiment1_results.json')

## Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Stage 1: HDC Dimension
ax = axes[0, 0]
dims = CONFIG['hdc_dims']
for model_key in ['distilbert', 'gpt2']:
    accs = [stage1_results[model_key]['hdc_results'][d] for d in dims]
    ax.plot(dims, accs, 'o-', label=model_key, markersize=8)
ax.axhline(stage0_results['distilbert'], ls='--', alpha=0.5, label='distilbert ceiling')
ax.axhline(stage0_results['gpt2'], ls='--', alpha=0.5, label='gpt2 ceiling')
ax.set_xlabel('HDC Dimension')
ax.set_ylabel('Accuracy')
ax.set_title('Stage 1: HDC Dimension Effect')
ax.legend()
ax.set_ylim(0.7, 0.95)

# Stage 2: Alignment Methods
ax = axes[0, 1]
methods = ['none', 'procrustes', 'cca', 'contrastive']
colors = ['gray', 'orange', 'green', 'red']
for method, color in zip(methods, colors):
    means, stds = [], []
    for anchor_size in CONFIG['anchor_sizes']:
        accs = [e['accuracy'] for e in stage2_results['experiments'] 
               if e['method'] == method and e['anchor_size'] == anchor_size]
        means.append(np.mean(accs))
        stds.append(np.std(accs))
    ax.errorbar(CONFIG['anchor_sizes'], means, yerr=stds, marker='o', 
               label=method, color=color, capsize=4)
ax.axhline(stage2_results['student_ceiling'], ls='--', color='green', alpha=0.5, label='Student ceiling')
ax.set_xlabel('Anchor Size')
ax.set_ylabel('Accuracy')
ax.set_title('Stage 2: Alignment Methods')
ax.legend()
ax.set_ylim(0.4, 0.85)

# Stage 3: Model Pairs
ax = axes[1, 0]
labels = [f"{p['teacher']}→{p['student']}" for p in stage3_results['pairs']]
ceilings = [p['student_ceiling'] for p in stage3_results['pairs']]
transfers = [p['mean_accuracy'] for p in stage3_results['pairs']]
x = np.arange(len(labels))
ax.bar(x - 0.2, ceilings, 0.35, label='Student ceiling', alpha=0.7)
ax.bar(x + 0.2, transfers, 0.35, label='Transfer', alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=15)
ax.set_ylabel('Accuracy')
ax.set_title('Stage 3: Model Pairs')
ax.legend()
ax.set_ylim(0, 1.0)

# Stage 4: Datasets
ax = axes[1, 1]
datasets = list(stage4_results['datasets'].keys())
pair1_accs = [stage4_results['datasets'][d]['pairs'][0]['mean_accuracy'] for d in datasets]
pair2_accs = [stage4_results['datasets'][d]['pairs'][1]['mean_accuracy'] for d in datasets]
x = np.arange(len(datasets))
ax.bar(x - 0.2, pair1_accs, 0.35, label='distilbert→gpt2', alpha=0.7)
ax.bar(x + 0.2, pair2_accs, 0.35, label='gpt2→distilbert', alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels([d.upper() for d in datasets])
ax.set_ylabel('Accuracy')
ax.set_title('Stage 4: Different Datasets')
ax.legend()
ax.set_ylim(0, 1.0)

plt.tight_layout()
plt.savefig('paper1_experiment1_results.png', dpi=150, bbox_inches='tight')
plt.show()

print('Visualization saved!')

## Summary

In [ ]:
print('=' * 60)
print('EXPERIMENT 1 SUMMARY')
print('=' * 60)

print('\nStage 0 - Model Ceilings:')
for model, acc in stage0_results.items():
    print(f'  {model}: {acc:.1%}')

print('\nStage 1 - HDC Cost (8192d):')
for model in ['distilbert', 'gpt2']:
    ceiling = stage0_results[model]
    hdc = stage1_results[model]['hdc_results'][8192]
    print(f'  {model}: {ceiling:.1%} → {hdc:.1%} (loss: {(hdc-ceiling)*100:.1f}%)')

print('\nStage 2 - Alignment Methods (500 anchors, mean):')
for method in ['none', 'procrustes', 'cca', 'contrastive']:
    accs = [e['accuracy'] for e in stage2_results['experiments'] 
           if e['method'] == method and e['anchor_size'] == 500]
    print(f'  {method}: {np.mean(accs):.1%}')

print('\nStage 3 - Model Pairs:')
for p in stage3_results['pairs']:
    print(f"  {p['teacher']}→{p['student']}: {p['mean_accuracy']:.1%} ({p['mean_efficiency']:.1%} eff)")

print('\nStage 4 - Datasets:')
for ds_name, ds_data in stage4_results['datasets'].items():
    acc = np.mean([p['mean_accuracy'] for p in ds_data['pairs']])
    eff = np.mean([p['mean_efficiency'] for p in ds_data['pairs']])
    print(f'  {ds_name}: {acc:.1%} ({eff:.1%} eff)')